In [1]:
import numpy as np
import os
from pathlib import Path
paths_list=[]
labels_list=[]
for i in range (1, 110):
    if i != 100 and i != 92 and i != 88 :
        for j in range(3, 15):
            for r in range(0, 400):
                if os.path.isfile(f"EEG MI ROI-F/datasets/S{str(i).rjust(3, '0')}R{str(j).rjust(2, '0')}{str(r).rjust(3, '0')}.npy"):
                    paths_list.append(f"EEG MI ROI-F/datasets/S{str(i).rjust(3, '0')}R{str(j).rjust(2, '0')}{str(r).rjust(3, '0')}.npy")
labels_list = np.load('EEG MI ROI-F/labels_list.npy')
print(len(paths_list))
print(len(labels_list))
# print(paths_list)

343323
343323


In [2]:
from sklearn import model_selection
def train_test_split(series_list, labels_list):
    x_train, x_test, y_train, y_test = model_selection.train_test_split(
        series_list, labels_list, test_size=0.125, random_state=42, shuffle=True
    )
    return x_train, x_test, y_train, y_test
x_train, x_test, y_train, y_test = train_test_split(paths_list, labels_list)

In [3]:
import tensorflow as tf
import math
class EMMDSequence(tf.keras.utils.Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        low = idx * self.batch_size
        # Cap upper bound at array length; the last batch may be smaller
        # if the total number of items is not a multiple of batch size.
        high = min(low + self.batch_size, len(self.x))
        batch_x = self.x[low:high]
        batch_y = self.y[low:high]

        return np.array([np.load(file_name) for file_name in batch_x]), np.array(batch_y)

2023-09-04 16:58:01.474271: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-04 16:58:02.165833: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
from tensorflow.keras.layers import Conv1D,MaxPooling1D,BatchNormalization,Dense,SpatialDropout1D,Flatten,Input,SeparableConv1D,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
from tensorflow.keras import optimizers
num_classes = 5
sgd=optimizers.SGD(learning_rate=0.1)
def SimplifiedCNN():
    model = Sequential()
    clear_session()
    model.add(Conv1D(25, 11, strides=1, padding='valid', activation="relu" ,input_shape = (640,2)))
    model.add(BatchNormalization())
    model.add(SpatialDropout1D(0.25))
    
    model.add(Conv1D(25, 1, strides=1, padding='valid', activation="relu"))
    model.add(MaxPooling1D(pool_size=3, strides=3))
    model.add(BatchNormalization())

    model.add(Conv1D(50, 11, strides=1, padding='valid', activation="relu"))
    model.add(MaxPooling1D(pool_size=3, strides=3))
    model.add(BatchNormalization())
    model.add(SpatialDropout1D(0.25))
    
    model.add(Conv1D(100, 11, strides=1, padding='valid', activation="relu"))
    model.add(MaxPooling1D(pool_size=3, strides=3))
    model.add(BatchNormalization())
    model.add(SpatialDropout1D(0.25))
    
    model.add(Conv1D(200, 11, strides=1, padding='valid', activation="relu"))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = SimplifiedCNN()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 630, 25)           575       
                                                                 
 batch_normalization (Batch  (None, 630, 25)           100       
 Normalization)                                                  
                                                                 
 spatial_dropout1d (Spatial  (None, 630, 25)           0         
 Dropout1D)                                                      
                                                                 
 conv1d_1 (Conv1D)           (None, 630, 25)           650       
                                                                 
 max_pooling1d (MaxPooling1  (None, 210, 25)           0         
 D)                                                              
                                                        

In [7]:
training_generator = EMMDSequence(x_train, y_train, 512)
validation_generator = EMMDSequence(x_test, y_test, 512)
model.fit(training_generator, epochs=50, validation_data=validation_generator) 

Epoch 1/50
587/587 [==============================] - 89s 147ms/step - loss: 1.2202 - accuracy: 0.4859 - val_loss: 1.1150 - val_accuracy: 0.5209
Epoch 2/50
587/587 [==============================] - 57s 96ms/step - loss: 1.1250 - accuracy: 0.5223 - val_loss: 1.0806 - val_accuracy: 0.5425
Epoch 3/50
587/587 [==============================] - 56s 96ms/step - loss: 1.1007 - accuracy: 0.5356 - val_loss: 1.0588 - val_accuracy: 0.5566
Epoch 4/50
587/587 [==============================] - 57s 98ms/step - loss: 1.0837 - accuracy: 0.5442 - val_loss: 1.0558 - val_accuracy: 0.5546
Epoch 5/50
587/587 [==============================] - 57s 96ms/step - loss: 1.0697 - accuracy: 0.5495 - val_loss: 1.0410 - val_accuracy: 0.5641
Epoch 6/50
587/587 [==============================] - 56s 95ms/step - loss: 1.0590 - accuracy: 0.5555 - val_loss: 1.0302 - val_accuracy: 0.5717
Epoch 7/50
587/587 [==============================] - 56s 95ms/step - loss: 1.0495 - accuracy: 0.5607 - val_loss: 1.0242 - val_accuracy

In [8]:
model.save('202309041640')

INFO:tensorflow:Assets written to: 202309041640/assets


INFO:tensorflow:Assets written to: 202309041640/assets


In [9]:
model.fit(training_generator, epochs=50, validation_data=validation_generator) 

Epoch 1/50
587/587 [==============================] - 60s 101ms/step - loss: 0.7936 - accuracy: 0.6781 - val_loss: 0.8474 - val_accuracy: 0.6582
Epoch 2/50
587/587 [==============================] - 58s 98ms/step - loss: 0.7918 - accuracy: 0.6797 - val_loss: 0.8488 - val_accuracy: 0.6550
Epoch 3/50
587/587 [==============================] - 58s 99ms/step - loss: 0.7878 - accuracy: 0.6804 - val_loss: 0.8482 - val_accuracy: 0.6549
Epoch 4/50
587/587 [==============================] - 58s 99ms/step - loss: 0.7841 - accuracy: 0.6834 - val_loss: 0.8415 - val_accuracy: 0.6595
Epoch 5/50
587/587 [==============================] - 58s 98ms/step - loss: 0.7811 - accuracy: 0.6842 - val_loss: 0.8399 - val_accuracy: 0.6603
Epoch 6/50
587/587 [==============================] - 59s 101ms/step - loss: 0.7802 - accuracy: 0.6845 - val_loss: 0.8420 - val_accuracy: 0.6594
Epoch 7/50
587/587 [==============================] - 57s 98ms/step - loss: 0.7747 - accuracy: 0.6869 - val_loss: 0.8426 - val_accurac

In [10]:
model.save('202309041922')

INFO:tensorflow:Assets written to: 202309041922/assets


INFO:tensorflow:Assets written to: 202309041922/assets


In [11]:
model.fit(training_generator, epochs=50, validation_data=validation_generator) 

Epoch 1/50
587/587 [==============================] - 64s 109ms/step - loss: 0.6963 - accuracy: 0.7217 - val_loss: 0.7908 - val_accuracy: 0.6847
Epoch 2/50
587/587 [==============================] - 56s 95ms/step - loss: 0.6947 - accuracy: 0.7230 - val_loss: 0.7921 - val_accuracy: 0.6838
Epoch 3/50
587/587 [==============================] - 56s 95ms/step - loss: 0.6949 - accuracy: 0.7222 - val_loss: 0.7912 - val_accuracy: 0.6842
Epoch 4/50
587/587 [==============================] - 57s 97ms/step - loss: 0.6940 - accuracy: 0.7228 - val_loss: 0.7930 - val_accuracy: 0.6824
Epoch 5/50
587/587 [==============================] - 58s 98ms/step - loss: 0.6927 - accuracy: 0.7228 - val_loss: 0.7905 - val_accuracy: 0.6853
Epoch 6/50
587/587 [==============================] - 58s 98ms/step - loss: 0.6896 - accuracy: 0.7235 - val_loss: 0.7907 - val_accuracy: 0.6820
Epoch 7/50
587/587 [==============================] - 56s 96ms/step - loss: 0.6904 - accuracy: 0.7242 - val_loss: 0.7899 - val_accuracy

In [12]:
model.save('202309042011')

INFO:tensorflow:Assets written to: 202309042011/assets


INFO:tensorflow:Assets written to: 202309042011/assets
